In [8]:
# !pip install hanja
# !pip install mxnet
# !pip install gluonnlp pandas tqdm
# !pip install sentencepiece
# !pip install transformers
# !pip install torch
# !pip install hanja
# !pip install imblearn
# !pip install WordCloud
# !pip install seaborn 

In [1]:
from transformers import BertTokenizerFast, AlbertModel, BertModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
#import hanja
from imblearn.over_sampling import RandomOverSampler
from transformers.optimization import get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup
from transformers import AdamW
from collections import defaultdict

import gc
from tqdm import tqdm


In [2]:
tokenizer_bert_kor_base = BertTokenizerFast.from_pretrained("kykim/albert-kor-base")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


In [3]:
# electra-kor-base
from transformers import ElectraTokenizerFast, ElectraModel, AdamW, ElectraForSequenceClassification
# tokenizer : kykim/bert-kor-base

def preprocessing(title):
  x = title[:]
  x = re.sub('><', ' ', x)
  x = re.sub('[ㄱ-ㅎ!@#$%^&*(),./?1-9]', ' ', x)
  return x.strip()

class KLUEDataset_train(Dataset):
  def __init__(self, DataFrame) :   
    self.dataset = DataFrame
    self.tokenizer = tokenizer

    # print(self.dataset.describe())

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.loc[idx, ['title', 'topic_idx']].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=64,
        pad_to_max_length=True,
        add_special_tokens=True
    )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y
class KLUEDataset_test(Dataset):
  def __init__(self, DataFrame):
    self.dataset = DataFrame
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.loc[idx, ['title']].values
    text = row[0]

    inputs = self.tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=64,
        pad_to_max_length=True,
        add_special_tokens=True
    )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask


In [4]:
PRE_TRAINED_MODEL_NAME = "kykim/electra-kor-base"
tokenizer = ElectraTokenizerFast.from_pretrained(PRE_TRAINED_MODEL_NAME)
device = torch.device("cuda")


In [4]:
df = pd.read_table('data/한국어욕설데이터.txt', sep='|', names=['sentence','lbl'])
df = df[df['lbl'] == 0]
df = df.dropna()
df['sentence'] = df['sentence'].apply(preprocessing)
df = df[df['sentence'] != ""]

df['범죄'] = 0
df['혐오'] = 0
df['선정'] = 0
df['폭력'] = 0
df['비난'] = 0
df['욕설'] = 0
df['차별'] = 0
df['lbl'] = 1
df.columns = ['sentence', '비도덕아님','범죄','혐오','선정','폭력','비난','욕설','차별']
#pd.concat([origin_train, df])

In [23]:

# Train data
origin_train = pd.read_csv('data/train_ml2.csv')
origin_train = origin_train.dropna()
origin_train = origin_train.loc[:,['sentence', '범죄', '혐오', '선정', '비도덕 아님', '폭력', '비난', '욕설', '차별']]
origin_train.columns = ['sentence', '범죄', '혐오', '선정', '비도덕아님', '폭력', '비난', '욕설', '차별']
origin_train = pd.concat([origin_train, df]) # df는 한국어욕설데이터
#origin_train['sentence'] = origin_train['sentence'].apply(preprocessing)

# Test data
origin_test = pd.read_csv("data/test_ml2.csv")
origin_test = origin_test.loc[:,['sentence', '범죄', '혐오', '선정', '비도덕 아님', '폭력', '비난', '욕설', '차별']]
origin_test.columns = ['sentence', '범죄', '혐오', '선정', '비도덕아님', '폭력', '비난', '욕설', '차별']
#test['sentence'] = test['sentence'].apply(preprocessing)

label = ['범죄', '혐오', '선정', '비도덕아님', '폭력', '비난', '욕설', '차별']

for lbl in label : 
    
    train = origin_train.copy()
    test = origin_test.copy()
    # 모델 조건에 맞추고, lbl 에 맞추어서 필터링
    titles_t = train.sentence.to_numpy().reshape(-1,1)
    labels_t = train[lbl].to_numpy().reshape(-1,1)
    train = pd.DataFrame({'title':titles_t.reshape(-1), 'topic_idx':labels_t.reshape(-1)})

    titles_t = test.sentence.to_numpy().reshape(-1,1)
    labels_t = test[lbl].to_numpy().reshape(-1,1)
    df_valid = pd.DataFrame({'title':titles_t.reshape(-1), 'topic_idx':labels_t.reshape(-1)})

    # 위의 모델과 다른 부분
    train_dataset = KLUEDataset_train(train)
    test_dataset = KLUEDataset_test(df_valid)
    gc.collect()
    torch.cuda.empty_cache()
    EPOCHS = 10
    batch_size = 64
    train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_data_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    total_steps = len(train_data_loader) * EPOCHS

    # 모델 학습 파라미터
    warmup_ratio = 0.1
    NUM_OF_MODELS = 1
    import random
    losses = []
    accuracy = []

    model = ElectraForSequenceClassification.from_pretrained("kykim/electra-kor-base", num_labels=2).to(device)
    optimizer = AdamW(model.parameters(), lr=1e-4)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=1, num_training_steps=total_steps)

    for i in range(EPOCHS):
        total_loss = 0.0
        correct = 0
        total = 0
        batches = 0

        model.train()

        for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_data_loader):
            optimizer.zero_grad()
            y_batch = y_batch.to(device)
            y_batch = y_batch.long()
            y_pred = model(input_ids_batch.to(device), attention_mask = attention_masks_batch.to(device))[0]
            loss = F.cross_entropy(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()

            _, predicted = torch.max(y_pred, 1)
            correct += (predicted == y_batch).sum()
            total += len(y_batch)

            batches += 1
            if batches % 100 == 0:
                print("Batch Loss: ", total_loss, "Accuracy: ", correct.float() / total)
            
        losses.append(total_loss)
        accuracy.append(correct.float() / total)
        print("Train Loss :", total_loss, "Accuracy :", correct.float() / total)

    answer = []
    # torch.save(model, './Electra_kor_base.model')
    model.eval()

    for input_ids_batch, attention_masks_batch in tqdm(test_data_loader):
        y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
        _, predicted = torch.max(y_pred, 1)
        answer.append(predicted.item())
            
    globals()['{}'.format(lbl)] = answer
    gc.collect()
    torch.cuda.empty_cache()


Some weights of the model checkpoint at kykim/electra-kor-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at kykim/electra-kor-base and are newly initialized: ['classifier.out_proj.weight', 'clas

Batch Loss:  13.394590187817812 Accuracy:  tensor(0.9681, device='cuda:0')


 93%|█████████▎| 200/215 [00:57<00:04,  3.48it/s]

Batch Loss:  22.990321865305305 Accuracy:  tensor(0.9745, device='cuda:0')


100%|██████████| 215/215 [01:01<00:00,  3.52it/s]


...

 93%|█████████▎| 200/215 [00:56<00:04,  3.54it/s]

Batch Loss:  16.317902948707342 Accuracy:  tensor(0.9841, device='cuda:0')


100%|██████████| 215/215 [01:01<00:00,  3.52it/s]


Train Loss : 17.985238322988153 Accuracy : tensor(0.9836, device='cuda:0')


100%|██████████| 1038/1038 [00:20<00:00, 51.29it/s]


In [24]:

# origin_test = pd.read_csv("test_ml2.csv")
# origin_test = origin_test.loc[:,['sentence', '범죄', '혐오', '선정', '비도덕 아님', '폭력', '비난', '욕설', '차별']]
# origin_test.columns = ['sentence', '범죄', '혐오', '선정', '비도덕아님', '폭력', '비난', '욕설', '차별']

# kykim/electra-kor-base
model_name = "electra-kor-base"

test_label = []
for idx, row in origin_test.iloc[:, [1,2,3,4,5,6,7,8]].iterrows() :
  test_label.append(list(row))

# 각 모델별 결과를 합친 것 비교해보기
count = 0
for i, data in enumerate(zip(범죄, 혐오, 선정, 비도덕아님, 폭력, 비난, 욕설, 차별)) :
    if list(data) == test_label[i] :
      count +=1
print(count)

accuracy = count / len(origin_test)
print("8개 Label 예측 accuracy :", accuracy)

result = {
    "범죄" : 범죄, 
    "혐오" : 혐오, 
    "선정" : 선정, 
    "비도덕아님" : 비도덕아님, 
    "폭력" : 폭력, 
    "비난" : 비난, 
    "욕설" : 욕설, 
    "차별" : 차별
}
result = pd.DataFrame(result)
result.to_csv("result/label8_{}.csv".format(model_name))


# 비도덕아님은 예측하지 않게 진행
# 비도덕아님은 다른 케이스가 모두 False 인 경우
t = []
for i, data in enumerate(zip(범죄, 혐오, 선정, 폭력, 비난, 욕설, 차별)) :
    if sum(list(data)) == 0  :
        t.append(1)
    else :
        t.append(0)


count = 0
for i, data in enumerate(zip(범죄, 혐오, 선정, t, 폭력, 비난, 욕설, 차별)) :
    if list(data) == test_label[i] :
      count +=1
print(count)

accuracy = count / len(origin_test)
print("7개 Label 예측 accuracy :", accuracy)      

result = pd.DataFrame(result)
result.to_csv("result/label7_{}.csv".format(model_name))


425
8개 Label 예측 accuracy : 0.4094412331406551
474
7개 Label 예측 accuracy : 0.45664739884393063


In [17]:
j=0
c=0
for i in test_label :
    if i[0] == 범죄[j] :
        c += 1
print(c/1038)

0.9903660886319846


In [ ]:
# 12.11 에 와서 -> 이게 바뀌는지 봐야함.
# 535
# 8개 Label 예측 accuracy : 0.5154142581888247
# 666
# 7개 Label 예측 accuracy : 0.6416184971098265

In [8]:
# 데이터 저장 형식
result[:5]

,범죄,혐오,선정,비도덕아님,폭력,비난,욕설,차별
0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,1,0,0
2,0,0,0,0,0,1,0,0
3,0,0,0,0,0,1,0,0
4,0,0,0,0,0,1,0,0


In [23]:
# 모델 저장하는 방법 -> 용량이 너무 큼
# model_name = 'Electra_kor_base'
# torch.save(model, "model/{}_{}.pt".format(model_name,lbl))

In [24]:
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

sh: 0: getcwd() failed: No such file or directory
The folder you are executing pip from can no longer be found.


In [36]:

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return len(self.labels)
    
    

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
import gc

In [37]:
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model


device = torch.device("cuda")
bertmodel, vocab = get_pytorch_kobert_model()



# Train data
origin_train = pd.read_csv('train_ml2.csv')
origin_train = origin_train.dropna()
origin_train = origin_train.loc[:,['sentence', '범죄', '혐오', '선정', '비도덕 아님', '폭력', '비난', '욕설', '차별']]
origin_train.columns = ['sentence', '범죄', '혐오', '선정', '비도덕아님', '폭력', '비난', '욕설', '차별']
#origin_train['sentence'] = origin_train['sentence'].apply(preprocessing)

# Test data
origin_test = pd.read_csv("test_ml2.csv")
origin_test = origin_test.loc[:,['sentence', '범죄', '혐오', '선정', '비도덕 아님', '폭력', '비난', '욕설', '차별']]
origin_test.columns = ['sentence', '범죄', '혐오', '선정', '비도덕아님', '폭력', '비난', '욕설', '차별']
#test['sentence'] = test['sentence'].apply(preprocessing)

label = ['범죄', '혐오', '선정', '비도덕아님', '폭력', '비난', '욕설', '차별']

# label이 8개 이렇게 되어있고

max_len = 64 # 
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 50
learning_rate =  1e-5


for lbl in label : 
        
    
    train = origin_train.copy()
    test = origin_test.copy()
    
    titles_t = train.sentence.to_numpy().reshape(-1,1)
    labels_t = train[lbl].to_numpy().reshape(-1,1)
    train = pd.DataFrame({'title':titles_t.reshape(-1), 'topic_idx':labels_t.reshape(-1)})

    titles_t = test.sentence.to_numpy().reshape(-1,1)
    labels_t = test[lbl].to_numpy().reshape(-1,1)
    df_valid = pd.DataFrame({'title':titles_t.reshape(-1), 'topic_idx':labels_t.reshape(-1)})
    
    dataset = []
    for title, label in zip(train.title.to_list(), train.topic_idx.to_list()):
      dataset.append([title,label])
    dataset_train = dataset[:].copy()
    dataset_test = random.sample(dataset,2000).copy() # random 으로 2000개 뽑게하기

    len(dataset_train), len(dataset_test)

    tokenizer = get_tokenizer()
    tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

## Setting parameters
    data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
    data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

    train_dataloader = torch.utils.data.DataLoader(data_train,  shuffle=True,batch_size=batch_size, num_workers=4)
    test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

    gc.collect()
    torch.cuda.empty_cache()

    valid_dataset = []
    for title, label in zip(df_valid.title.to_list(), np.zeros(len(df_valid))):
      valid_dataset.append([title,label])
    real_test_data = BERTDataset(valid_dataset, 0, 1, tok, max_len, True, False)
    real_test_dataloader = torch.utils.data.DataLoader(real_test_data, batch_size=1, num_workers=4)



    NUM_OF_MODELS = 1
    import random
    for NUM in range(NUM_OF_MODELS):
      dr_r = 0.5
      print("epoch: ",NUM+1)
      print("drop out rate: ",dr_r)
      model = BERTClassifier(bertmodel,  dr_rate=dr_r ).to(device)
      # Prepare optimizer and schedule (linear warmup and decay)
      no_decay = ['bias', 'LayerNorm.weight']
      optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
      ]
      optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
      loss_fn = nn.CrossEntropyLoss()
      t_total = len(train_dataloader) * num_epochs
      warmup_step = int(t_total * warmup_ratio)
      scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
      for e in range(num_epochs):
          train_acc = 0.0
          test_acc = 0.0
          model.train()
          for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
              optimizer.zero_grad()
              token_ids = token_ids.long().to(device)
              segment_ids = segment_ids.long().to(device)
              valid_length= valid_length
              label = label.long().to(device)
              out = model(token_ids, valid_length, segment_ids)
              loss = loss_fn(out, label)
              loss.backward()
              torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
              optimizer.step()
              scheduler.step()  # Update learning rate schedule
              train_acc += calc_accuracy(out, label)
              if batch_id % log_interval == 0:
                  print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
          print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
          model.eval()
          for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
              token_ids = token_ids.long().to(device)
              segment_ids = segment_ids.long().to(device)
              valid_length= valid_length
              label = label.long().to(device)
              out = model(token_ids, valid_length, segment_ids)
              test_acc += calc_accuracy(out, label)
          print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
      answer=[]
      model.eval()
      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(real_test_dataloader)):
              token_ids = token_ids.long().to(device)
              segment_ids = segment_ids.long().to(device)
              valid_length= valid_length
              label = label.long().to(device)
              out = model(token_ids, valid_length, segment_ids)
              max_vals, max_indices = torch.max(out, 1)
              answer.append(max_indices[0].item())
      globals()['{}'.format(lbl)] = answer
      gc.collect()
      torch.cuda.empty_cache()

using cached model
using cached model
using cached model
epoch:  1
drop out rate:  0.5


/tmp/ipykernel_27680/1338714451.py:101: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/157 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/tmp/ipykernel_27680/1338714451.py:118: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/32 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/1038 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [38]:

# origin_test = pd.read_csv("test_ml2.csv")
# origin_test = origin_test.loc[:,['sentence', '범죄', '혐오', '선정', '비도덕 아님', '폭력', '비난', '욕설', '차별']]
# origin_test.columns = ['sentence', '범죄', '혐오', '선정', '비도덕아님', '폭력', '비난', '욕설', '차별']

# kykim/electra-kor-base
model_name = "kobert"

test_label = []
for idx, row in origin_test.iloc[:, [1,2,3,4,5,6,7,8]].iterrows() :
  test_label.append(list(row))

# 각 모델별 결과를 합친 것 비교해보기
count = 0
for i, data in enumerate(zip(범죄, 혐오, 선정, 비도덕아님, 폭력, 비난, 욕설, 차별)) :
    if list(data) == test_label[i] :
      count +=1
print(count)

accuracy = count / len(origin_test)
print("8개 Label 예측 accuracy :", accuracy)

result = {
    "범죄" : 범죄, 
    "혐오" : 혐오, 
    "선정" : 선정, 
    "비도덕아님" : 비도덕아님, 
    "폭력" : 폭력, 
    "비난" : 비난, 
    "욕설" : 욕설, 
    "차별" : 차별
}
result = pd.DataFrame(result)
result.to_csv("predict_result/max64_label8_{}.csv".format(model_name))


# 비도덕아님은 예측하지 않게 진행
# 비도덕아님은 다른 케이스가 모두 False 인 경우
t = []
for i, data in enumerate(zip(범죄, 혐오, 선정, 폭력, 비난, 욕설, 차별)) :
    if sum(list(data)) == 0  :
        t.append(1)
    else :
        t.append(0)


count = 0
for i, data in enumerate(zip(범죄, 혐오, 선정, t, 폭력, 비난, 욕설, 차별)) :
    if list(data) == test_label[i] :
      count +=1
print(count)

accuracy = count / len(origin_test)
print("7개 Label 예측 accuracy :", accuracy)      

result = pd.DataFrame(result)
result.to_csv("predict_result/max64_abel7_{}.csv".format(model_name))


591
8개 Label 예측 accuracy : 0.569364161849711
658
7개 Label 예측 accuracy : 0.6339113680154143


In [7]:
class KLUEDataset_train(Dataset):
  def __init__(self, DataFrame):
    self.dataset = DataFrame
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

    # print(self.dataset.describe())

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.loc[idx, ['title', 'topic_idx']].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=64,
        pad_to_max_length=True,
        add_special_tokens=True
    )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y
class KLUEDataset_test(Dataset):
  def __init__(self, DataFrame):
    self.dataset = DataFrame
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
    # print(self.dataset.describe())

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.loc[idx, ['title']].values
    text = row[0]

    inputs = self.tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=32,
        pad_to_max_length=True,
        add_special_tokens=True
    )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask

In [49]:
# koelectra


# Train data
origin_train = pd.read_csv('train_ml2.csv')
origin_train = origin_train.dropna()
origin_train = origin_train.loc[:,['sentence', '범죄', '혐오', '선정', '비도덕 아님', '폭력', '비난', '욕설', '차별']]
origin_train.columns = ['sentence', '범죄', '혐오', '선정', '비도덕아님', '폭력', '비난', '욕설', '차별']
origin_train = pd.concat([origin_train, df]) # df는 한국어욕설데이터

#origin_train['sentence'] = origin_train['sentence'].apply(preprocessing)

# Test data
origin_test = pd.read_csv("test_ml2.csv")
origin_test = origin_test.loc[:,['sentence', '범죄', '혐오', '선정', '비도덕 아님', '폭력', '비난', '욕설', '차별']]
origin_test.columns = ['sentence', '범죄', '혐오', '선정', '비도덕아님', '폭력', '비난', '욕설', '차별']
#test['sentence'] = test['sentence'].apply(preprocessing)

label = ['범죄', '혐오', '선정', '비도덕아님', '폭력', '비난', '욕설', '차별']

device = torch.device("cuda")
bertmodel, vocab = get_pytorch_kobert_model()

# 모델 학습 파라미터
epochs = 10
batch_size = 64
warmup_ratio=0.1

for lbl in label : 


    train = origin_train.copy()
    test = origin_test.copy()
    # 모델 조건에 맞추고, lbl 에 맞추어서 필터링
    titles_t = train.sentence.to_numpy().reshape(-1,1)
    labels_t = train[lbl].to_numpy().reshape(-1,1)
    train = pd.DataFrame({'title':titles_t.reshape(-1), 'topic_idx':labels_t.reshape(-1)})

    titles_t = test.sentence.to_numpy().reshape(-1,1)
    labels_t = test[lbl].to_numpy().reshape(-1,1)
    df_valid = pd.DataFrame({'title':titles_t.reshape(-1), 'topic':labels_t.reshape(-1)})


    train_dataset = KLUEDataset_train(train)
    test_dataset = KLUEDataset_test(df_valid)


    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    t_total = len(train_loader) * epochs
    gc.collect()
    torch.cuda.empty_cache()

    losses = []
    accuracy = []
    model = ElectraForSequenceClassification.from_pretrained('monologg/koelectra-base-v3-discriminator', num_labels=2).to(device)
    optimizer = AdamW(model.parameters(), lr=1e-4)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=1, num_training_steps=t_total)

    for i in range(epochs):
      total_loss = 0.0
      correct = 0
      total = 0
      batches = 0

      model.train()

      for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        optimizer.zero_grad()
        y_batch = y_batch.to(device)
        y_batch = y_batch.long()
        y_pred = model(input_ids_batch.to(device), attention_mask = attention_masks_batch.to(device))[0]
        loss = F.cross_entropy(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

        _, predicted = torch.max(y_pred, 1)
        correct += (predicted == y_batch).sum()
        total += len(y_batch)

        batches += 1
        if batches % 100 == 0:
          print("Batch Loss: ", total_loss, "Accuracy: ", correct.float() / total)
          
      losses.append(total_loss)
      accuracy.append(correct.float() / total)
      print("Train Loss :", total_loss, "Accuracy :", correct.float() / total)

    answer = []
    model.eval()
    for input_ids_batch, attention_masks_batch in tqdm(test_loader):
      y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
      _, predicted = torch.max(y_pred, 1)
      answer.append(predicted.item())

    globals()['{}'.format(lbl)] = answer

using cached model
using cached model


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

Batch Loss:  15.288895254954696 Accuracy:  tensor(0.9639, device='cuda:0')


100%|██████████| 157/157 [00:43<00:00,  3.59it/s]


Train Loss : 22.593319612555206 Accuracy : tensor(0.9668, device='cuda:0')


...

 64%|██████▎   | 100/157 [00:27<00:15,  3.59it/s]

Batch Loss:  10.530230436474085 Accuracy:  tensor(0.9781, device='cuda:0')


100%|██████████| 157/157 [00:43<00:00,  3.61it/s]


Train Loss : 16.96262983419001 Accuracy : tensor(0.9775, device='cuda:0')


100%|██████████| 1038/1038 [00:13<00:00, 77.60it/s]


In [50]:

model_name = "ko-electra"

test_label = []
for idx, row in origin_test.iloc[:, [1,2,3,4,5,6,7,8]].iterrows() :
  test_label.append(list(row))

# 각 모델별 결과를 합친 것 비교해보기
count = 0
for i, data in enumerate(zip(범죄, 혐오, 선정, 비도덕아님, 폭력, 비난, 욕설, 차별)) :
    if list(data) == test_label[i] :
      count +=1
print(count)

accuracy = count / len(origin_test)
print("8개 Label 예측 accuracy :", accuracy)

result = {
    "범죄" : 범죄, 
    "혐오" : 혐오, 
    "선정" : 선정, 
    "비도덕아님" : 비도덕아님, 
    "폭력" : 폭력, 
    "비난" : 비난, 
    "욕설" : 욕설, 
    "차별" : 차별
}
result = pd.DataFrame(result)
result.to_csv("predict_result/label8_{}.csv".format(model_name))


# 비도덕아님은 예측하지 않게 진행
# 비도덕아님은 다른 케이스가 모두 False 인 경우
t = []
for i, data in enumerate(zip(범죄, 혐오, 선정, 폭력, 비난, 욕설, 차별)) :
    if sum(list(data)) == 0  :
        t.append(1)
    else :
        t.append(0)


count = 0
for i, data in enumerate(zip(범죄, 혐오, 선정, t, 폭력, 비난, 욕설, 차별)) :
    if list(data) == test_label[i] :
      count +=1
print(count)

accuracy = count / len(origin_test)
print("7개 Label 예측 accuracy :", accuracy)      

result = pd.DataFrame(result)
result.to_csv("predict_result/label7_{}.csv".format(model_name))


603
8개 Label 예측 accuracy : 0.5809248554913294
720
7개 Label 예측 accuracy : 0.6936416184971098


In [5]:
# koelectra v1

class KLUEDataset_train(Dataset):
  def __init__(self, DataFrame):
    self.dataset = DataFrame
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-discriminator")

    # print(self.dataset.describe())

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.loc[idx, ['title', 'topic_idx']].values
    text = row[0]
    y = row[1]
    inputs = self.tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=64,
        pad_to_max_length=True,
        add_special_tokens=True
    )
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]
    return input_ids, attention_mask, y
class KLUEDataset_test(Dataset):
  def __init__(self, DataFrame):
    self.dataset = DataFrame
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-discriminator")
    # print(self.dataset.describe())

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.loc[idx, ['title']].values
    text = row[0]

    inputs = self.tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=64,
        pad_to_max_length=True,
        add_special_tokens=True
    )
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]
    return input_ids, attention_mask

In [6]:
# koelectra v1
from transformers import ElectraTokenizerFast, ElectraModel, AdamW, ElectraForSequenceClassification


import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model


device = torch.device("cuda")
bertmodel, vocab = get_pytorch_kobert_model()

# Train data
origin_train = pd.read_csv('data/train_ml2.csv')
origin_train = origin_train.dropna()
origin_train = origin_train.loc[:,['sentence', '범죄', '혐오', '선정', '비도덕 아님', '폭력', '비난', '욕설', '차별']]
origin_train.columns = ['sentence', '범죄', '혐오', '선정', '비도덕아님', '폭력', '비난', '욕설', '차별']
origin_train = pd.concat([origin_train, df]) # df는 한국어욕설데이터

#origin_train['sentence'] = origin_train['sentence'].apply(preprocessing)

# Test data
origin_test = pd.read_csv("data/test_ml2.csv")
origin_test = origin_test.loc[:,['sentence', '범죄', '혐오', '선정', '비도덕 아님', '폭력', '비난', '욕설', '차별']]
origin_test.columns = ['sentence', '범죄', '혐오', '선정', '비도덕아님', '폭력', '비난', '욕설', '차별']
#test['sentence'] = test['sentence'].apply(preprocessing)

device = torch.device("cuda")
bertmodel, vocab = get_pytorch_kobert_model()

# 모델 학습 파라미터
epochs = 10
batch_size = 64
warmup_ratio=0.1

label = ['범죄', '혐오', '선정', '비도덕아님', '폭력', '비난', '욕설', '차별']

for lbl in label : 

    train = origin_train.copy()
    test = origin_test.copy()
    # 모델 조건에 맞추고, lbl 에 맞추어서 필터링
    titles_t = train.sentence.to_numpy().reshape(-1,1)
    labels_t = train[lbl].to_numpy().reshape(-1,1)
    train = pd.DataFrame({'title':titles_t.reshape(-1), 'topic_idx':labels_t.reshape(-1)})

    titles_t = test.sentence.to_numpy().reshape(-1,1)
    labels_t = test[lbl].to_numpy().reshape(-1,1)
    df_valid = pd.DataFrame({'title':titles_t.reshape(-1), 'topic':labels_t.reshape(-1)})



    train_dataset = KLUEDataset_train(train)
    test_dataset = KLUEDataset_test(df_valid)
    # 모델 학습 파라미터

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    t_total = len(train_loader) * epochs
    losses = []
    accuracy = []
    model = ElectraForSequenceClassification.from_pretrained('monologg/koelectra-base-discriminator', num_labels=2).to(device)
    optimizer = AdamW(model.parameters(), lr=1e-4)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=1, num_training_steps=t_total)
    for i in range(epochs):
      total_loss = 0.0
      correct = 0
      total = 0
      batches = 0

      model.train()
      for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        optimizer.zero_grad()
        y_batch = y_batch.to(device)
        y_batch = y_batch.long()
        y_pred = model(input_ids_batch.to(device), attention_mask = attention_masks_batch.to(device))[0]
        loss = F.cross_entropy(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

        _, predicted = torch.max(y_pred, 1)
        correct += (predicted == y_batch).sum()
        total += len(y_batch)

        batches += 1
        if batches % 100 == 0:
          print("Batch Loss: ", total_loss, "Accuracy: ", correct.float() / total)
          
      losses.append(total_loss)
      accuracy.append(correct.float() / total)
      print("Train Loss :", total_loss, "Accuracy :", correct.float() / total)

    answer = []
    model.eval()
    for input_ids_batch, attention_masks_batch in tqdm(test_loader):
      y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
      _, predicted = torch.max(y_pred, 1)
      answer.append(predicted.item())
      globals()['{}'.format(lbl)] = answer
# 이거 테스트중이었음

using cached model
using cached model
using cached model
using cached model


Some weights of the model checkpoint at monologg/koelectra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-discriminator and are newly initialized: ['clas

Batch Loss:  12.777726512402296 Accuracy:  tensor(0.9600, device='cuda:0')


 93%|█████████▎| 200/215 [01:46<00:08,  1.83it/s]

Batch Loss:  21.900274272076786 Accuracy:  tensor(0.9702, device='cuda:0')


...

 93%|█████████▎| 200/215 [00:56<00:04,  3.55it/s]

Batch Loss:  16.621037179604173 Accuracy:  tensor(0.9838, device='cuda:0')


100%|██████████| 215/215 [01:00<00:00,  3.54it/s]


Train Loss : 18.018944900482893 Accuracy : tensor(0.9836, device='cuda:0')


100%|██████████| 1038/1038 [00:20<00:00, 51.69it/s]


In [8]:

model_name = "ko-electra-v1"

test_label = []
for idx, row in origin_test.iloc[:, [1,2,3,4,5,6,7,8]].iterrows() :
  test_label.append(list(row))

# 각 모델별 결과를 합친 것 비교해보기
count = 0
for i, data in enumerate(zip(범죄, 혐오, 선정, 비도덕아님, 폭력, 비난, 욕설, 차별)) :
    if list(data) == test_label[i] :
      count +=1
print(count)

accuracy = count / len(origin_test)
print("8개 Label 예측 accuracy :", accuracy)

result = {
    "범죄" : 범죄, 
    "혐오" : 혐오, 
    "선정" : 선정, 
    "비도덕아님" : 비도덕아님, 
    "폭력" : 폭력, 
    "비난" : 비난, 
    "욕설" : 욕설, 
    "차별" : 차별
}
result = pd.DataFrame(result)
result.to_csv("result/test_label8_{}.csv".format(model_name))


# 비도덕아님은 예측하지 않게 진행
# 비도덕아님은 다른 케이스가 모두 False 인 경우
t = []
for i, data in enumerate(zip(범죄, 혐오, 선정, 폭력, 비난, 욕설, 차별)) :
    if sum(list(data)) == 0  :
        t.append(1)
    else :
        t.append(0)


count = 0
for i, data in enumerate(zip(범죄, 혐오, 선정, t, 폭력, 비난, 욕설, 차별)) :
    if list(data) == test_label[i] :
      count +=1
print(count)

accuracy = count / len(origin_test)
print("7개 Label 예측 accuracy :", accuracy)      

result = pd.DataFrame(result)
result.to_csv("result/test_label7_{}.csv".format(model_name))


615
8개 Label 예측 accuracy : 0.5924855491329479
705
7개 Label 예측 accuracy : 0.6791907514450867


In [11]:
pred = []
for i, data in enumerate(zip(범죄, 혐오, 선정, t, 폭력, 비난, 욕설, 차별)) :
    pred.append(data)

In [1]:
from sklearn.metrics import f1_score
f1_score(test_label, pred, average='micro')



NameError: name 'test_label' is not defined

In [26]:
data2 = pd.read_csv('unethical_ml.csv')
test_label = []
for idx, row in data2.iloc[:, [1,2,3,4,5,6,7,8]].iterrows() :
  test_label.append(list(row))

# 각 모델별 결과를 합친 것 비교해보기
count = 0
for i, data in enumerate(zip(범죄, 혐오, 선정, 비도덕아님, 폭력, 비난, 욕설, 차별)) :
    if list(data) == test_label[i] :
      count +=1
print(count)

accuracy = count / len(data)
print("accuracy :", accuracy)


373
accuracy : 46.625


In [57]:
# KoElectra_v2

class KLUEDataset_train(Dataset):
  def __init__(self, DataFrame):
    self.dataset = DataFrame
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v2-discriminator")


  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.loc[idx, ['title', 'topic_idx']].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=64,
        pad_to_max_length=True,
        add_special_tokens=True
    )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y
class KLUEDataset_test(Dataset):
  def __init__(self, DataFrame):
    self.dataset = DataFrame
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v2-discriminator")

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.loc[idx, ['title']].values
    text = row[0]

    inputs = self.tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=64,
        pad_to_max_length=True,
        add_special_tokens=True
    )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]
    return input_ids, attention_mask

In [58]:
# KoElectra_v2

label = ['범죄', '혐오', '선정', '비도덕아님', '폭력', '비난', '욕설', '차별']


# Train data
origin_train = pd.read_csv('train_ml2.csv')
origin_train = origin_train.dropna()
origin_train = origin_train.loc[:,['sentence', '범죄', '혐오', '선정', '비도덕 아님', '폭력', '비난', '욕설', '차별']]
origin_train.columns = ['sentence', '범죄', '혐오', '선정', '비도덕아님', '폭력', '비난', '욕설', '차별']
#origin_train['sentence'] = origin_train['sentence'].apply(preprocessing)

# Test data
origin_test = pd.read_csv("test_ml2.csv")
origin_test = origin_test.loc[:,['sentence', '범죄', '혐오', '선정', '비도덕 아님', '폭력', '비난', '욕설', '차별']]
origin_test.columns = ['sentence', '범죄', '혐오', '선정', '비도덕아님', '폭력', '비난', '욕설', '차별']
#test['sentence'] = test['sentence'].apply(preprocessing)

device = torch.device("cuda")
bertmodel, vocab = get_pytorch_kobert_model()

# 모델 학습 파라미터
epochs = 10
batch_size = 64
warmup_ratio=0.1


for lbl in label : 


    train = origin_train.copy()
    test = origin_test.copy()
    # 모델 조건에 맞추고, lbl 에 맞추어서 필터링
    titles_t = train.sentence.to_numpy().reshape(-1,1)
    labels_t = train[lbl].to_numpy().reshape(-1,1)
    train = pd.DataFrame({'title':titles_t.reshape(-1), 'topic_idx':labels_t.reshape(-1)})

    titles_t = test.sentence.to_numpy().reshape(-1,1)
    labels_t = test[lbl].to_numpy().reshape(-1,1)
    df_valid = pd.DataFrame({'title':titles_t.reshape(-1), 'topic':labels_t.reshape(-1)})



    train_dataset = KLUEDataset_train(train)
    test_dataset = KLUEDataset_test(df_valid)
    # 모델 학습 파라미터

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    t_total = len(train_loader) * epochs
    losses = []
    accuracy = []
    model = ElectraForSequenceClassification.from_pretrained('monologg/koelectra-base-v2-discriminator', num_labels=2).to(device)
    optimizer = AdamW(model.parameters(), lr=1e-4)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=1, num_training_steps=t_total)
    for i in range(epochs):
      total_loss = 0.0
      correct = 0
      total = 0
      batches = 0

      model.train()
      for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        optimizer.zero_grad()
        y_batch = y_batch.to(device)
        y_batch = y_batch.long()        
        y_pred = model(input_ids_batch.to(device), attention_mask = attention_masks_batch.to(device))[0]
        loss = F.cross_entropy(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

        _, predicted = torch.max(y_pred, 1)
        correct += (predicted == y_batch).sum()
        total += len(y_batch)

        batches += 1
        if batches % 100 == 0:
          print("Batch Loss: ", total_loss, "Accuracy: ", correct.float() / total)
          
      losses.append(total_loss)
      accuracy.append(correct.float() / total)
      print("Train Loss :", total_loss, "Accuracy :", correct.float() / total)

    answer = []
    model.eval()
    for input_ids_batch, attention_masks_batch in tqdm(test_loader):
      y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
      _, predicted = torch.max(y_pred, 1)
      answer.append(predicted.item())
      globals()['{}'.format(lbl)] = answer


using cached model
using cached model


Some weights of the model checkpoint at monologg/koelectra-base-v2-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v2-discriminator and are newly initialized: 

Batch Loss:  14.530044578015804 Accuracy:  tensor(0.9702, device='cuda:0')


100%|██████████| 157/157 [00:43<00:00,  3.59it/s]


Train Loss : 21.95918634533882 Accuracy : tensor(0.9703, device='cuda:0')


...

 64%|██████▎   | 100/157 [00:27<00:15,  3.59it/s]

Batch Loss:  19.0667005777359 Accuracy:  tensor(0.9336, device='cuda:0')


100%|██████████| 157/157 [00:43<00:00,  3.60it/s]


Train Loss : 29.7562470510602 Accuracy : tensor(0.9352, device='cuda:0')


  8%|▊         | 12/157 [00:03<00:40,  3.56it/s]

In [29]:
model_name = "ko-electra-v2"

test_label = []
for idx, row in origin_test.iloc[:, [1,2,3,4,5,6,7,8]].iterrows() :
  test_label.append(list(row))

# 각 모델별 결과를 합친 것 비교해보기
count = 0
for i, data in enumerate(zip(범죄, 혐오, 선정, 비도덕아님, 폭력, 비난, 욕설, 차별)) :
    if list(data) == test_label[i] :
      count +=1
print(count)

accuracy = count / len(origin_test)
print("8개 Label 예측 accuracy :", accuracy)

result = {
    "범죄" : 범죄, 
    "혐오" : 혐오, 
    "선정" : 선정, 
    "비도덕아님" : 비도덕아님, 
    "폭력" : 폭력, 
    "비난" : 비난, 
    "욕설" : 욕설, 
    "차별" : 차별
}
result = pd.DataFrame(result)
result.to_csv("predict_result/label8_{}.csv".format(model_name))


# 비도덕아님은 예측하지 않게 진행
# 비도덕아님은 다른 케이스가 모두 False 인 경우
t = []
for i, data in enumerate(zip(범죄, 혐오, 선정, 폭력, 비난, 욕설, 차별)) :
    if sum(list(data)) == 0  :
        t.append(1)
    else :
        t.append(0)


count = 0
for i, data in enumerate(zip(범죄, 혐오, 선정, t, 폭력, 비난, 욕설, 차별)) :
    if list(data) == test_label[i] :
      count +=1
print(count)

accuracy = count / len(origin_test)
print("7개 Label 예측 accuracy :", accuracy)      

result = pd.DataFrame(result)
result.to_csv("predict_result/label7_{}.csv".format(model_name))


376
accuracy : 47.0


In [ ]:
from sklearn.metrics import f1_score

y_true # 정답레이블 -> np.array(y_true)
result # 예측한 애
f1_score(y_true, result, average='micro')